In [95]:
import itertools
import numpy as np
from collections import Counter
import multiprocessing

PROCESSES = 4

In [96]:
DIRS = (-1, 1)

def transformations():
    for v1, v2 in itertools.permutations(np.eye(3, 3, dtype=int), 2):
        for d1, d2 in itertools.product(DIRS, DIRS):
            v3 = np.cross(v1*d1, v2*d2)
            yield np.stack([v1*d1, v2*d2, v3], axis=1)

ts = list(transformations())
MAX_DIST_1D = 1000

def parse_scans(s):
    scanners = s.split('\n\n')
    return [np.array(sorted([[int(p) for p in l.split(',')] for l in s.splitlines()[1:]]), dtype=int)
            for s in scanners]

def points_in_common(p1s, p2s):
    return (p1s[:, None] == p2s).all(2).any(1).sum()

def match_points(p1s, p2s, min_matches=12):
    for (i, p1), (j, p2) in itertools.product(enumerate(p1s), enumerate(p2s)):
        if i + min_matches > len(p1s): break
        translation = p1 - p2
        #if np.absolute(translation).sum() > MAX_DIST_1D*6: continue
        q2s = p2s + translation
        if points_in_common(p1s, q2s) >= min_matches:
            return translation, q2s

def transform_match_points(p1s, p2s, min_matches=12):
    for t in ts:
        match = match_points(p1s, p2s @ t, min_matches)
        if match is not None:
            return match

def map_beacons(scans, min_matches=12):
    corrected = {0: scans[0]}
    scanners = {0: np.zeros(3, dtype=int)}
    visited = set()
    
    while len(corrected) != len(scans):
        for i in corrected.keys() - visited:
            p1s = corrected[i]
            with multiprocessing.Pool(PROCESSES) as pool:
                params = [(j, p1s, p2s, min_matches) for (j, p2s) in enumerate(scans)
                          if j not in corrected]
                results = [pool.apply_async(transform_match_points, p[1:]) for p in params]
                for (j, *_), result in zip(params, results):
                    match = result.get()
                    if match is not None:
                        print('matched', i, j)
                        corrected[j] = match[1]
                        scanners[j] = match[0]
            visited.add(i)
    
    beacons = {tuple(p) for p in itertools.chain(*corrected.values())}
    return beacons, scanners

def manhattan_dist(q, r):
    return np.absolute(q - r).sum()

In [97]:
len(ts)

24

In [98]:
scans = parse_scans("""--- scanner 0 ---
0,2
4,1
3,3

--- scanner 1 ---
-1,-1
-5,0
-2,1""")
scans

[array([[0, 2],
        [3, 3],
        [4, 1]]),
 array([[-5,  0],
        [-2,  1],
        [-1, -1]])]

In [99]:
match_points(scans[0], scans[1], 3)

(array([5, 2]),
 array([[0, 2],
        [3, 3],
        [4, 1]]))

In [100]:
scans = parse_scans("""--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14""")

In [101]:
%%time
beacons, scanners = map_beacons(scans, 6)
len(beacons)

matched 0 1
matched 0 4
matched 1 2
matched 1 3
CPU times: user 30.4 ms, sys: 36.4 ms, total: 66.7 ms
Wall time: 654 ms


79

In [93]:
%%time
with open('../data/day19.txt') as infile:
    scans = parse_scans(infile.read())
    beacons, scanners = map_beacons(scans)
    print('[p1] Num beacons:', len(beacons))
    print('[p2] Max Manhattan dist:', max(manhattan_dist(s1, s2) for (s1, s2)
                                          in itertools.combinations(scanners.values(), 2)))

matched 0 3
matched 0 7
matched 0 32
matched 0 35
matched 32 4
matched 32 17
matched 32 19
matched 32 28
matched 3 23
matched 35 1
matched 35 6
matched 35 11
matched 35 14
matched 7 36
matched 1 22
matched 1 27
matched 6 5
matched 6 12
matched 6 37
matched 36 2
matched 36 29
matched 11 25
matched 17 8
matched 17 34
matched 28 30
matched 5 18
matched 8 21
matched 12 15
matched 29 31
matched 30 9
matched 30 26
matched 37 24
matched 9 10
matched 9 16
matched 21 20
matched 26 13
matched 20 33
[p1] Num beacons: 428
[p2] Max Manhattan dist: 12140
CPU times: user 1.73 s, sys: 1.1 s, total: 2.82 s
Wall time: 1min 1s
